# Caderno 5 - Aplica doc2query

## 1. Carrega os documentos para aplicar o doc2query

In [15]:
import pandas as pd

PASTA_DADOS = './dados/'
PASTA_RESULTADO_CADERNO = f'{PASTA_DADOS}outputs/5_doc2query/'
NOME_ARQUIVO_DOC2QUERY = f'{PASTA_RESULTADO_CADERNO}doc2query.pickle'

# A pasta dos JURIS aqui não é a pasta original, e sim o resultado do caderno 1 (os documentos já estão filtrados)
PASTA_JURIS_TCU = f'{PASTA_DADOS}outputs/1_tratamento_juris_tcu/'

# Carrega os arquivos 
def carrega_juris_tcu():
    doc1 = pd.read_csv(f'{PASTA_JURIS_TCU}doc_tratado_parte_1.csv', sep='|')
    doc2 = pd.read_csv(f'{PASTA_JURIS_TCU}doc_tratado_parte_2.csv', sep='|')
    doc3 = pd.read_csv(f'{PASTA_JURIS_TCU}doc_tratado_parte_3.csv', sep='|')
    doc4 = pd.read_csv(f'{PASTA_JURIS_TCU}doc_tratado_parte_4.csv', sep='|')
    doc = pd.concat([doc1, doc2, doc3, doc4], ignore_index=True)
    query = pd.read_csv(f'{PASTA_JURIS_TCU}query_tratado.csv', sep='|')
    qrel = pd.read_csv(f'{PASTA_JURIS_TCU}qrel_tratado.csv', sep='|')

    return doc, query, qrel

docs, _, _ = carrega_juris_tcu()

## 2. Carrega o modelo e aloca na GPU/CPU. Função para gerar as queries.

Peguei o exemplo direto do site do modelo e adaptei: https://huggingface.co/doc2query/msmarco-portuguese-mt5-base-v1

O ideal era aplicar isso em batch.

In [16]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
import torch

device = 'cuda' if torch.cuda.is_available() else 'cpu'

model_name = 'doc2query/msmarco-portuguese-mt5-base-v1'
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

model.to(device)

MT5ForConditionalGeneration(
  (shared): Embedding(250112, 768)
  (encoder): MT5Stack(
    (embed_tokens): Embedding(250112, 768)
    (block): ModuleList(
      (0): MT5Block(
        (layer): ModuleList(
          (0): MT5LayerSelfAttention(
            (SelfAttention): MT5Attention(
              (q): Linear(in_features=768, out_features=768, bias=False)
              (k): Linear(in_features=768, out_features=768, bias=False)
              (v): Linear(in_features=768, out_features=768, bias=False)
              (o): Linear(in_features=768, out_features=768, bias=False)
              (relative_attention_bias): Embedding(32, 12)
            )
            (layer_norm): MT5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): MT5LayerFF(
            (DenseReluDense): MT5DenseGatedActDense(
              (wi_0): Linear(in_features=768, out_features=2048, bias=False)
              (wi_1): Linear(in_features=768, out_features=2048, bias=False)
         

In [17]:
def doc2query(texto, random_sampling=True, num_return_sequences=5, top_p=0.95, top_k=10, max_length=64, seed=None):
    input_ids = tokenizer.encode(texto, return_tensors='pt').to(device)

    if seed is not None:
        torch.manual_seed(seed)
    
    with torch.no_grad():
        # Here we use top_k / top_k random sampling. It generates more diverse queries, but of lower quality
        if random_sampling:
            outputs = model.generate(
                input_ids=input_ids,
                max_length=max_length,
                do_sample=True,
                top_p=top_p,
                top_k=top_k,
                num_return_sequences=num_return_sequences
            )
        else:
            # Here we use Beam-search. It generates better quality queries, but with less diversity
            outputs = model.generate(
                input_ids=input_ids, 
                max_length=max_length, 
                num_beams=num_return_sequences, 
                no_repeat_ngram_size=2, 
                num_return_sequences=num_return_sequences, 
                early_stopping=True
            )
        
    queries = [tokenizer.decode(out, skip_special_tokens=True) for out in outputs]
    return queries

## 3. Aplica doc2query

Gera um exemplo qualquer:

In [18]:
text = "Não caracteriza marco interruptivo da prescrição das pretensões punitiva e ressarcitória do TCU ato de investigação dos fatos que não contém medidas inequívocas de apuração de condutas individualmente descritas e imputadas ao responsável."
doc2query(text, random_sampling=True, num_return_sequences=5, top_p=0.95, top_k=10, max_length=64, seed=42)

['o que significa marco interruptivo',
 'o que é um marco interruptivo da presscrição?',
 'definição do marco interruptivo de prescrição de hipotecas',
 'o que é um marco interruptivo da prescrição',
 'definir tcu marco interruptivo']

Agora aplica em toda a base:

In [19]:
import re
import pickle
from tqdm import tqdm

def remove_html(html):
    return re.sub("<[^>]*>", "", html).strip()

# Seta uma seed, por questões de reprudicibilidade
torch.manual_seed(42)

# Objeto que guardará as queries geradas por documento
queries_por_doc = {}

# Percorre o dataframe de ocumentos e gera as queries
for i, row in tqdm(docs.iterrows(), total=len(docs)):
    doc_key = row.KEY
    enunciado = remove_html(row.ENUNCIADO)
    queries_geradas = doc2query(enunciado)
    queries_por_doc[doc_key] = queries_geradas

# Salva num arquivo pickle
with open(NOME_ARQUIVO_DOC2QUERY, 'wb') as f:
    pickle.dump(queries_por_doc, f)

100%|██████████| 16045/16045 [2:01:32<00:00,  2.20it/s]  
